### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [27]:
SCRIPT_NAME = "Y2017M09D13_RH_EE_task_control_V01"
OUTPUT_VERSION = 1
OUTPUT_FILE_NAME = "detailed_tasks"


ec2_output_path = ("/volumes/data/{}/output_V{:02.0f}/").format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Output s3: " + s3_output_path +
      "\nOutput ec2: " + ec2_output_path)


Output s3: s3://wri-projects/Aqueduct30/processData/Y2017M09D13_RH_EE_task_control_V01/output_V01/
Output ec2: /volumes/data/Y2017M09D13_RH_EE_task_control_V01/output_V01/


In [16]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

Y2018M04D23 UTC 07:26


# Settings

In [2]:
MAXTASKS = 3000
CANCELTASKS = 0 # Cancels all pending tasks

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print(task)
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print('canceling %s' % task)
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print(i)
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
type(taskList)

list

In [8]:
len(taskList)

17144

In [ ]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


'/volumes/data/Y2017M09D13_RH_EE_task_control_V01/output_V01/detailed_tasks.csv'

In [20]:
detailedTasks.to_csv(ec2_output_path + OUTPUT_FILE_NAME + ".csv")
detailedTasks.to_pickle(ec2_output_path + OUTPUT_FILE_NAME + ".pkl")

In [29]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

upload: ../../../../data/Y2017M09D13_RH_EE_task_control_V01/output_V01/detailed_tasks.csv to s3://wri-projects/Aqueduct30/processData/Y2017M09D13_RH_EE_task_control_V01/output_V01/detailed_tasks.csv
upload: ../../../../data/Y2017M09D13_RH_EE_task_control_V01/output_V01/detailed_tasks.pkl to s3://wri-projects/Aqueduct30/processData/Y2017M09D13_RH_EE_task_control_V01/output_V01/detailed_tasks.pkl


# DANGER ZONE

In [12]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,id,output_url,progress,queuetime(min),runtime(min),start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,NaN,1524438818738,global_historical_PLivWN_month_m_5min_1960_201...,27BT6NEK45ZWZ6XYFLCLPWDL,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438818738
1,NaN,1524438815003,global_historical_PLivWN_month_m_5min_1960_201...,6DUEXXEB5TTYOBDI7QLG3KYX,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438815003
2,NaN,1524438811261,global_historical_PLivWN_month_m_5min_1960_201...,Z7OI6P3LDZSAXQCBYOEN3XGP,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438811261
3,NaN,1524438807554,global_historical_PLivWN_month_m_5min_1960_201...,HP6EOKA3YSBWJGPYL7TWNXSC,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438807554
4,NaN,1524438659599,global_historical_PLivWN_month_m_5min_1960_201...,L2IBCWUB3D27IJDICMMU2KNE,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438659599
5,NaN,1524438655899,global_historical_PLivWN_month_m_5min_1960_201...,EDFL6IO7VS3AO42N4T6THD2K,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438655899
6,NaN,1524438652216,global_historical_PLivWN_month_m_5min_1960_201...,EFVG4MBINVNIBGWKGQ2WE2RU,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438652216
7,NaN,1524438627579,global_historical_PLivWN_month_m_5min_1960_201...,7AHSZV7M3MO2V7RY45GSIRQG,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438627579
8,NaN,1524438624388,global_historical_PLivWN_month_m_5min_1960_201...,F237ECOLH6BQCUUQQM3WLTZC,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438624388
9,NaN,1524438621225,global_historical_PLivWN_month_m_5min_1960_201...,TSEKDNPJ3YGOWXFKNHIU4WLW,NaN,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1524438621225


In [ ]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)